In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict

In [2]:
os.getcwd()

'c:\\Users\\samue\\OneDrive\\Documenti\\Studio\\Università\\Tesi\\Progetto\\Ricerca\\Script'

In [3]:
os.chdir("../SYMBOL")

Mapping Probe -> Ensembl

In [4]:
os.chdir('../AnnotationDB')

In [5]:
path = "../SYMBOL"

In [7]:
huex10st = pd.read_csv('huex10st.tsv', sep="\t")[['PROBEID', 'SYMBOL']]
huex10st = huex10st.dropna()

hgu133a2 = pd.read_csv('hgu133a2.tsv', sep="\t")[['PROBEID', 'SYMBOL']]
hgu133a2 = hgu133a2.dropna()

hgu133plus2 = pd.read_csv('hgu133plus2.tsv', sep="\t")[['PROBEID', 'SYMBOL']]
hgu133plus2 = hgu133plus2.dropna()

illuminaHumanV3 = pd.read_csv('illuminaHumanV3.tsv', sep="\t")[['PROBEID', 'SYMBOL']]
illuminaHumanV3 = illuminaHumanV3.dropna()

illuminaHumanV4 = pd.read_csv('illuminaHumanV4.tsv', sep="\t")[['PROBEID', 'SYMBOL']]
illuminaHumanV4 = illuminaHumanV4.dropna()

huge10st = pd.read_csv('huge10st.csv')[['PROBEID', 'SYMBOL']]
huge10st = huge10st.dropna()

mapList = [huex10st, hgu133a2, hgu133plus2, illuminaHumanV3, illuminaHumanV4, huge10st]

counter = Counter()

for s in mapList:
    unique_values = set(s['SYMBOL'])  # Consideriamo solo i valori unici di ogni Series
    counter.update(unique_values)

# Conversione in dizionario
result_dict = dict(counter)

probeMapDef = []

for probeMap in mapList:
    probeMap['Count'] = probeMap['SYMBOL'].map(result_dict)
    print(probeMap)
    idx_max = probeMap.groupby('PROBEID')['Count'].idxmax()
    probeMap = probeMap.loc[idx_max, ['PROBEID', 'SYMBOL']].reset_index(drop=True)
    probeMapDef.append(dict(zip(probeMap['PROBEID'], probeMap['SYMBOL'])))
    print(probeMap)

       PROBEID   SYMBOL  Count
0      2315554   TTLL10      4
1      2315554  MIR200B      2
2      2315554  MIR200A      2
3      2315554   MIR429      2
4      2315633  B3GALT6      5
...        ...      ...    ...
29017  7385552    TRPV1      6
29018  7385611   HPCAL1      6
29019  7385641   CLSTN2      5
29020  7385683    VPS41      6
29021  7385696     SEM1      6

[24528 rows x 3 columns]
       PROBEID   SYMBOL
0      2315554   TTLL10
1      2315633  B3GALT6
2      2315674   SCNN1D
3      2315739    PUSL1
4      2315894     VWA1
...        ...      ...
17512  7385552    TRPV1
17513  7385611   HPCAL1
17514  7385641   CLSTN2
17515  7385683    VPS41
17516  7385696     SEM1

[17517 rows x 2 columns]
                           PROBEID SYMBOL  Count
0                        1007_s_at   DDR1      6
1                        1007_s_at   DDR1      6
2                        1007_s_at   DDR1      6
3                        1007_s_at   DDR1      6
4                        1007_s_at   DDR1  

Dataset mapping

In [8]:
os.chdir("../Normalized")

In [9]:
def changeNameDuplicates(dataset):
    new_columns = []
    seen = {}

    # Itera sui nomi delle colonne e aggiungi un suffisso numerico se la colonna è duplicata
    for col in dataset.columns:
        if col in seen:
            seen[col] += 1
            new_columns.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            new_columns.append(col)

    dataset.columns = new_columns
    return dataset

def aggregateProbes(dataset):
    grouped_cols = defaultdict(list)
    for col in dataset.columns:
        prefix = col.split('_')[0]
        grouped_cols[prefix].append(col)

    cols_to_keep = []

    for prefix, cols in grouped_cols.items():
        if len(cols) > 1:
            mediane = dataset[cols].median()
            best_col = mediane.idxmax()
            cols_to_keep.append(best_col)
        else:
            cols_to_keep.append(cols[0])

    return dataset[cols_to_keep]
    

Mapping GSE41848

In [ ]:
gse41848 = pd.read_csv('gse41848_norm.csv')
gse41848.columns = ['SampleID'] + list([probeMapDef[0][int(probeId)] if int(probeId) in probeMapDef[0] else None for probeId in gse41848.columns[1:]])
gse41848 = gse41848.drop(columns=[col for col in gse41848.columns if col is None])
gse41848 = changeNameDuplicates(gse41848)
gse41848 = aggregateProbes(gse41848)
gse41848.columns = gse41848.columns.str.split('_').str[0]

print(gse41848.columns.is_unique)
print(gse41848.columns.str.contains("_").any())

gse41848.to_csv(path+'MappedNormalized/gse41848_mapped.csv', index=False)

gse41848

True
False


,SampleID,TTLL10,B3GALT6,SCNN1D,PUSL1,VWA1,ATAD3A,CALML6,PRKCZ,SKI,...,GJB5,LINC02593,PERM1,HES4,SFTPD,MALAT1,CCL2,HPCAL1,CLSTN2,VPS41
0,GSM1026056_600009.0001A.CEL,6.924730,7.258574,7.109393,7.591207,9.150376,8.577223,5.650476,7.692232,8.600898,...,5.540343,6.181064,6.461532,7.261087,6.160145,12.825805,5.152735,8.962564,5.913600,8.700867
1,GSM1026057_600009.0001B.CEL,7.018403,7.344919,7.196417,7.525381,9.402824,8.942788,5.667952,7.516144,8.805436,...,5.897996,6.493877,6.326495,7.634335,6.044389,13.005506,4.909259,9.060058,5.880681,8.375610
2,GSM1026058_41461.0001A.CEL,7.155700,7.642886,7.330345,7.575014,9.359444,8.545028,6.482900,7.670045,8.623814,...,6.405965,6.401589,6.478955,7.605589,6.324146,12.756152,5.144057,8.904020,5.990068,8.461716
3,GSM1026059_41462.0001A.CEL,6.197024,7.574891,6.840716,7.124627,8.492326,8.222351,6.659471,7.725480,8.513152,...,5.604886,5.980288,5.914122,7.057778,5.636053,13.204004,4.714175,8.768938,5.459300,8.649626
4,GSM1026060_600029.0001A.CEL,6.904426,7.186236,6.872494,7.111434,8.775060,8.408105,5.372005,7.485391,8.245529,...,5.477375,5.959854,6.132127,7.592647,5.725033,12.823587,6.127159,8.687341,5.712132,8.657192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,GSM1026263_9961.601495C.CEL,6.980864,7.424531,7.084429,7.467496,9.232243,8.441900,6.159012,7.745262,8.706905,...,5.454267,6.426737,6.230831,7.460203,6.119301,12.872699,5.171027,8.820246,5.687205,8.507256
208,GSM1026264_9961.601498B.CEL,6.918967,7.179801,7.039750,7.421819,8.963837,8.545584,6.347401,7.575038,8.542120,...,6.081252,6.309839,6.163265,7.479589,6.247839,13.020364,4.800163,8.876733,5.723460,8.559294
209,GSM1026265_9961.601498C.CEL,7.103975,7.243953,7.295930,7.264347,9.226749,8.615283,6.905716,7.624646,8.658098,...,5.596425,6.620260,6.485023,7.490335,6.367822,12.888698,5.139950,8.974267,5.931133,8.473372
210,GSM1026266_9961.601499B.CEL,6.736354,7.121445,6.918207,7.380354,8.870748,8.486787,6.169985,7.628387,8.772421,...,5.646759,6.077683,5.973958,7.283163,6.072895,12.892512,4.994554,9.170577,5.697745,8.510629


Mapping GSE41849

In [ ]:
gse41849 = pd.read_csv('gse41849_norm.csv')
gse41849.columns = ['SampleID'] + list([probeMapDef[0][int(probeId)] if int(probeId) in probeMapDef[0] else None for probeId in gse41849.columns[1:]])
gse41849 = gse41849.drop(columns=[col for col in gse41849.columns if col is None])
gse41849 = changeNameDuplicates(gse41849)
gse41849 = aggregateProbes(gse41849)
gse41849.columns = gse41849.columns.str.split('_').str[0]

print(gse41849.columns.is_unique)
print(gse41849.columns.str.contains("_").any())

gse41849.to_csv(path+'/MappedNormalized/gse41849_mapped.csv', index=False)

gse41849

True
False


,SampleID,TTLL10,B3GALT6,SCNN1D,PUSL1,VWA1,ATAD3A,CALML6,PRKCZ,SKI,...,GJB5,LINC02593,PERM1,HES4,SFTPD,MALAT1,CCL2,HPCAL1,CLSTN2,VPS41
0,GSM1026268_600001.0001A.CEL,6.465439,6.702136,6.308176,6.573874,8.130153,7.792286,4.724695,6.717893,7.737049,...,4.897351,5.585141,5.393523,6.686922,5.288890,12.640203,4.174754,7.676643,4.774791,8.274732
1,GSM1026269_600001.0001B.CEL,6.381168,6.726107,6.373549,6.750389,8.127751,7.656362,5.517173,6.732765,7.715124,...,5.106840,5.438379,4.984269,6.385045,4.992562,12.423562,3.806466,8.036265,4.895186,8.404932
2,GSM1026270_41495.0001A.CEL,6.383518,6.526469,6.421931,6.686922,8.213866,7.627505,5.957091,6.886517,7.789531,...,4.678670,5.628435,5.092142,6.362024,5.054772,12.632668,3.882308,7.713436,4.804700,8.538697
3,GSM1026271_41495.0001B.CEL,6.280446,6.325385,6.203353,6.502269,8.091896,7.375298,4.905678,6.547712,7.478863,...,4.602572,5.397481,5.075563,6.373776,4.868195,12.639994,3.688806,7.831258,4.730497,8.632235
4,GSM1026272_41495.0001C.CEL,5.965353,6.332898,5.982015,6.535255,8.019695,7.599663,5.889745,6.829910,7.522648,...,4.770036,5.080659,5.034817,6.142518,4.871044,12.461747,3.552906,7.951108,4.534071,8.688618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,GSM1026365_9961.601468B.CEL,6.500698,6.776298,6.386304,6.757979,8.192205,7.939494,5.555533,6.919854,7.683739,...,5.005992,5.527400,5.244237,6.568132,5.426630,12.387276,3.892635,7.993031,4.825320,8.428537
98,GSM1026366_9961.601497A.CEL,6.431804,6.702123,6.587386,6.777637,8.344335,7.875373,4.804120,6.816691,7.970997,...,4.671197,5.706907,5.419610,6.840581,5.132963,12.459493,3.956819,7.987539,4.852762,8.307896
99,GSM1026367_9961.601497B.CEL,6.539249,6.599873,6.593250,6.763405,8.263116,7.462055,5.537079,6.850474,7.832958,...,5.154167,5.467627,5.424302,6.758759,5.035335,12.287547,4.395262,7.832952,5.016946,8.202130
100,GSM1026368_9961.601503A.CEL,6.416943,6.776692,6.110979,6.346834,8.186136,7.620184,5.180170,6.684569,7.707520,...,4.968893,5.346835,5.368472,6.445257,5.183953,12.630142,3.682240,7.656333,4.797092,8.618942


Mapping GSE146383

In [ ]:
gse146383 = pd.read_csv('gse146383_norm.csv')
gse146383.columns = ['SampleID'] + list([probeMapDef[1][probeId] if probeId in probeMapDef[1] else None for probeId in gse146383.columns[1:]])
gse146383 = gse146383.drop(columns=[col for col in gse146383.columns if col is None])
gse146383 = changeNameDuplicates(gse146383)
gse146383 = aggregateProbes(gse146383)
gse146383.columns = gse146383.columns.str.split('_').str[0]

print(gse146383.columns.is_unique)
print(gse146383.columns.str.contains("_").any())

gse146383.to_csv(path+'/MappedNormalized/gse146383_mapped.csv', index=False)

gse146383

True
False


,SampleID,DDR1,RFC2,HSPA6,PAX8,GUCA1A,UBA7,THRA,PTPN21,CCL5,...,NUS1P3,PRR5-ARHGAP8,ZNF710-AS1,SAP25,RBM12B,FAM239A,TMEM231,SNHG17,HNRNPUL2-BSCL2,LOC100505915
0,GSM4379743_5532PDMS210_HG-U133A_2_.CEL,6.698221,6.622857,8.274699,6.992745,4.234660,8.916449,7.253199,4.323596,12.077566,...,4.237663,4.006649,6.637020,7.351526,5.627895,5.101655,3.986686,7.630125,4.525915,6.004914
1,GSM4379744_7618ADMS212.CEL,6.912708,6.671007,7.472181,7.223148,4.398831,8.888688,7.083353,4.285655,10.845859,...,4.005949,3.856791,6.469319,7.996352,5.432392,5.435531,3.678825,7.837500,4.652539,6.352938
2,GSM4379745_6893PDMS210_HG-U133A_2_.CEL,6.846882,6.568234,8.636458,6.862533,4.334546,8.797132,7.128009,3.976812,10.829543,...,4.035484,3.611328,6.439088,7.198997,5.686729,4.747545,3.518319,7.538745,4.966928,6.381636
3,GSM4379746_11283ADMS213.CEL,6.830051,6.532085,7.860873,7.262857,4.245650,8.556475,7.516863,4.258751,10.953007,...,4.522982,3.913162,6.647475,7.492011,5.133644,5.469290,3.456498,8.174182,4.915700,6.168750
4,GSM4379747_13183PDMS211.CEL,6.951035,6.865958,7.813450,7.071702,4.278833,8.549703,7.154154,4.079210,11.433185,...,4.394237,4.014210,6.605260,7.284666,5.957104,5.200485,3.378906,7.674055,5.103107,5.979047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,GSM4379823_3075C55.CEL,6.136122,6.889907,8.403068,7.123269,3.896777,8.689579,6.803494,3.813682,12.059535,...,3.877228,3.561333,5.960750,6.401378,6.404665,4.620227,2.943241,6.477389,4.135360,6.269250
81,GSM4379824_1970C44.CEL,6.288837,6.561245,8.694177,6.871955,4.053000,8.750633,6.588275,4.724892,11.535943,...,4.104080,3.551449,6.406675,6.641231,5.651598,4.631276,3.108007,6.967466,3.934854,6.616628
82,GSM4379825_2717C50.CEL,6.088990,6.569566,9.301647,6.855202,4.080702,8.436211,6.597867,4.089778,10.968543,...,3.779182,3.184109,6.128479,6.690341,7.147178,4.723442,2.889307,6.582310,4.622291,7.685432
83,GSM4379826_12952C191.CEL,6.134876,7.102008,8.733073,6.959534,4.047389,8.162031,6.538477,4.061891,11.787119,...,4.305047,4.037393,6.924333,6.492269,6.169961,4.577647,3.812579,7.387582,5.269773,6.308241


Mapping GSE13732

In [ ]:
gse13732 = pd.read_csv('gse13732_norm.csv')
gse13732.columns = ['SampleID'] + list([probeMapDef[2][probeId] if probeId in probeMapDef[2] else None for probeId in gse13732.columns[1:]])
gse13732 = gse13732.drop(columns=[col for col in gse13732.columns if col is None])
gse13732 = changeNameDuplicates(gse13732)
gse13732 = aggregateProbes(gse13732)
gse13732.columns = gse13732.columns.str.split('_').str[0]

print(gse13732.columns.is_unique)
print(gse13732.columns.str.contains("_").any())

gse13732.to_csv(path+'/MappedNormalized/gse13732_mapped.csv', index=False)

gse13732

True
False


,SampleID,DDR1,RFC2,HSPA6,PAX8,GUCA1A,UBA7,THRA,PTPN21,CCL5,...,ITGB1-DT,TNFRSF10A-DT,LOC400499,GALR3,NUS1P3,ZNF710-AS1,SAP25,TMEM231,HNRNPUL2-BSCL2,LOC100505915
0,GSM345077.CEL,6.446610,5.099553,7.275993,7.314208,5.734945,9.906071,7.466683,4.782265,10.409320,...,5.169539,5.327017,6.331004,6.944287,2.704381,7.892243,8.291651,4.735852,4.008973,4.892570
1,GSM345078.CEL,7.008966,4.911590,7.283107,7.274120,5.755652,9.857332,7.639633,4.849266,10.263477,...,4.956010,5.010572,6.867261,6.742351,2.448952,8.086546,9.122068,3.828395,3.941116,4.874534
2,GSM345079.CEL,7.030311,5.399787,6.444297,7.309508,5.677595,9.868690,7.713109,4.910870,11.688482,...,4.848790,5.387588,6.555606,6.891969,2.591603,8.352127,9.019494,4.152852,4.219170,5.058268
3,GSM345080.CEL,6.799201,5.537612,5.830621,7.406119,5.600025,9.021786,8.702302,4.336696,9.386464,...,4.912323,5.266130,6.298123,7.092312,3.285116,7.896886,8.009994,4.282668,4.065609,5.140872
4,GSM345081.CEL,7.581627,5.186826,6.888516,7.692765,5.806041,10.301989,6.935147,4.990862,9.054405,...,5.276206,5.250870,6.804209,6.647132,2.537166,7.649429,10.532459,4.070344,3.968167,5.165741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,GSM345185.CEL,7.426603,5.600051,7.849954,7.594912,6.189319,10.768841,9.325174,4.855347,11.364945,...,4.988589,5.469298,7.033654,7.112071,2.555487,7.814885,8.967682,4.697047,4.132576,5.096699
109,GSM345186.CEL,7.227828,5.454085,6.477328,7.260952,5.851286,10.964479,9.503565,5.191422,10.449027,...,4.704282,5.374219,6.916688,7.078457,2.636483,7.821477,9.006412,4.706660,4.024385,5.183254
110,GSM345187.CEL,6.991978,5.611728,9.210102,7.221401,6.173692,10.941772,8.843465,5.331870,11.066933,...,5.142403,5.281472,6.961829,6.918755,2.743780,7.673055,8.978856,4.354934,4.108086,5.234136
111,GSM345188.CEL,7.126070,5.450849,7.829113,7.360792,5.972281,10.727988,9.373280,5.116855,11.441340,...,4.771269,5.326996,7.180287,6.943761,2.553820,7.828928,9.223414,4.215035,4.215756,5.361234


Mapping GSE136411

In [10]:
gse136411 = pd.read_csv('gse136411_norm.csv').T
gse136411.columns = gse136411.iloc[0]
gse136411 = gse136411.drop(gse136411.index[0])
gse136411.insert(0, 'Index', gse136411.index)
gse136411 = gse136411.reset_index(drop=True)
gse136411[gse136411.columns[1:]] = gse136411[gse136411.columns[1:]].replace(',', '.', regex=True).apply(pd.to_numeric, errors='coerce')
gse136411.columns = ['SampleID'] + list([probeMapDef[4][probeId] if probeId in probeMapDef[4] else None for probeId in gse136411.columns[1:]])
gse136411 = gse136411.drop(columns=[col for col in gse136411.columns if col is None])
gse136411 = changeNameDuplicates(gse136411)
gse136411 = aggregateProbes(gse136411)
gse136411.columns = gse136411.columns.str.split('_').str[0]

print(gse136411.columns.is_unique)
print(gse136411.columns.str.contains("_").any())

# gse136411.to_csv(path+'/MappedNormalized/gse136411_mapped.csv', index=False)

gse136411

True
False


,SampleID,EEF1A1,SLC35E2A,RPS28,IPO13,LPP,UGP2,HNRNPAB,BORCS5,SNIP1,...,CDIP1,ERCC2,NANS,LAMTOR1,HNRNPA1,ZNF112,GYPE,NUP107,MRM2,PCYOX1L
0,Sample1,15.657022,6.330206,14.776577,9.420818,11.075051,8.746244,12.425740,5.223301,9.414929,...,7.926334,9.157902,9.462531,11.104380,6.889232,6.357474,5.584516,9.565671,8.503776,9.284923
1,Sample2,15.561864,5.734589,15.078814,9.033049,11.982526,8.257524,12.027812,5.642928,9.197991,...,7.101042,7.783759,9.128262,11.167347,5.919506,6.454838,6.292224,10.597142,7.683980,10.258152
2,Sample3_v2_Rep1,15.657022,7.035492,14.403041,8.705113,11.888393,8.861514,12.039875,5.484414,9.588201,...,7.053603,7.816427,8.522848,11.457560,6.959892,6.230378,6.346331,10.332933,8.216284,8.250858
3,Sample3_v2_Rep2,15.538618,5.876560,14.766608,8.918990,12.022342,9.085029,12.230457,5.922848,9.399408,...,6.982796,7.444619,8.564611,11.013710,7.134507,5.882457,5.758181,10.103496,7.717214,9.042396
4,Sample4,15.511204,5.306196,14.969296,8.646503,11.714060,7.458075,11.986740,5.956690,9.415875,...,6.903918,7.781812,8.966714,11.113594,6.750332,6.572611,6.151217,9.788740,7.753584,9.708349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,Sample309,15.691081,6.314830,14.810181,8.460978,12.172395,7.836679,12.228027,5.055831,9.238568,...,6.915242,7.699022,8.761878,11.448259,7.001206,5.404925,6.191433,10.516095,7.837033,9.326020
332,Sample310,15.635515,6.157879,15.317801,7.700820,11.810025,6.910580,12.202392,5.229205,9.802521,...,5.738102,7.695302,8.724278,10.818523,7.178935,5.867064,6.469524,9.659393,8.107990,9.255257
333,Sample311,15.586999,6.901912,14.606430,9.030786,11.821805,8.162239,12.214143,5.206721,9.527082,...,7.033110,7.949509,8.652347,10.800630,7.362643,6.508378,6.076818,10.203556,7.777209,8.938660
334,Sample312,15.531707,6.078865,14.328423,8.710531,12.225676,7.793092,12.610489,5.097750,9.538349,...,6.948147,7.683521,8.631449,10.422898,7.403477,6.156894,6.516539,10.364740,7.825902,9.498042


Mapping GSE17048

In [ ]:
gse17048 = pd.read_csv('gse17048_norm.csv').T
gse17048.columns = gse17048.iloc[0]
gse17048 = gse17048.drop(gse17048.index[0])
gse17048.insert(0, 'Index', gse17048.index)
gse17048 = gse17048.reset_index(drop=True)
gse17048.columns = ['SampleID'] + list([probeMapDef[3][probeId] if probeId in probeMapDef[3] else None for probeId in gse17048.columns[1:]])
gse17048 = gse17048.drop(columns=[col for col in gse17048.columns if col is None])
gse17048 = changeNameDuplicates(gse17048)
gse17048 = aggregateProbes(gse17048)
gse17048.columns = gse17048.columns.str.split('_').str[0]

print(gse17048.columns.is_unique)
print(gse17048.columns.str.contains("_").any())

gse17048.to_csv(path+'/MappedNormalized/gse17048_mapped.csv', index=False)

gse17048

True
False


,SampleID,MACC1,A1CF,POTEA,POTEB3,POTEE,POTEH,RBFOX1,A2M,A2ML1,...,ZUP1,ZW10,ZWILCH,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,C013,6.283466,6.034799,5.988436,5.980697,6.346561,6.297392,6.017639,5.74943,5.964265,...,6.486065,7.908795,6.422591,6.880596,8.064089,5.864531,10.366424,11.059243,9.004545,7.538623
1,G004R,5.890555,6.229092,6.545268,5.889097,6.27982,6.318432,6.065828,5.745214,6.078639,...,6.956828,8.245489,6.708988,6.753089,7.993903,5.945813,10.344611,11.378628,8.59763,7.815824
2,G038S,5.85568,6.096331,6.433919,6.296273,6.043136,6.235698,5.96882,5.880377,5.668971,...,6.953411,7.499508,6.611189,6.760061,7.656124,6.135658,10.439867,11.48864,8.810683,7.545563
3,C037,5.755901,6.146479,6.110024,5.810231,6.402523,6.42878,6.006056,6.012038,5.913057,...,6.684188,8.009858,6.481507,6.791497,8.195299,6.083444,10.324886,11.504691,8.902399,7.424471
4,P005,6.102647,6.158976,6.454089,5.825723,6.273007,6.331365,6.222459,6.242541,6.186799,...,6.816855,8.016697,6.548147,6.783563,7.749521,6.08474,10.171313,11.427389,8.753515,7.79653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,G027S,6.171776,6.141848,6.215416,6.182456,6.001572,6.319475,6.0475,6.088829,6.06917,...,7.120365,7.991584,6.354562,6.564407,7.813015,6.167559,10.446969,11.137138,9.068765,7.752917
140,P027,5.866225,6.238612,6.360454,6.143908,6.251569,6.234426,6.209482,6.075164,5.975168,...,7.077184,8.261232,6.388259,6.500589,7.762943,5.998673,9.864362,11.513161,9.117381,7.423872
141,C007,6.115124,6.081704,6.218627,6.236705,6.234909,6.128257,5.9987,6.130707,6.0203,...,7.439699,8.20759,6.667637,6.608756,7.663613,5.973435,9.814198,10.629927,8.637694,7.982452
142,P034,6.116467,6.310164,6.10983,6.048473,6.231421,6.255759,6.041616,5.78685,6.127446,...,6.94093,8.638221,6.7141,6.800555,7.843777,6.110461,10.123017,11.371261,9.048835,7.745126


Mapping GSE41890

In [80]:
gse41890 = pd.read_csv('gse41890_norm.csv')
gse41890.columns = ['SampleID'] + list([probeMapDef[5][int(probeId)] if int(probeId) in probeMapDef[5] else None for probeId in gse41890.columns[1:]])
gse41890 = gse41890.drop(columns=[col for col in gse41890.columns if col is None])
gse41890 = changeNameDuplicates(gse41890)
gse41890 = aggregateProbes(gse41890)
gse41890.columns = gse41890.columns.str.split('_').str[0]

print(gse41890.columns.is_unique)
print(gse41890.columns.str.contains("_").any())

gse41890.to_csv(path+'/MappedNormalized/gse41890_mapped.csv', index=False)

gse41890

True
False


,SampleID,ENSG00000177693,ENSG00000203880,ENSG00000284733,ENSG00000225972,ENSG00000177757,ENSG00000228794,ENSG00000187634,ENSG00000187961,ENSG00000187583,...,ENSG00000253953,ENSG00000233608,ENSG00000262612,ENSG00000270882,ENSG00000276600,ENSG00000124208,ENSG00000187134,ENSG00000244411,ENSG00000244094,ENSG00000243678
0,GSM1026993_00-024BM.CEL,7.218190,8.539534,8.149388,13.374557,5.271290,5.743456,6.538488,6.243999,6.563925,...,7.042150,7.131259,3.861512,9.918747,5.525397,10.382924,5.836659,8.539457,7.859374,8.763125
1,GSM1026994_00-024NBM.CEL,7.908879,9.269876,8.464631,13.388271,5.448298,5.505649,6.471197,6.300117,6.525126,...,6.908508,7.178301,4.391282,9.575293,5.641441,10.510459,5.832224,8.654092,7.971629,8.778608
2,GSM1026995_00-096BV.CEL,5.394984,9.177617,6.324851,13.391886,5.105521,6.519663,6.572430,6.472266,6.707904,...,6.911521,7.108358,3.842766,10.023421,5.417751,10.839924,5.536816,8.251293,7.783000,9.482312
3,GSM1026996_00-096NBV.CEL,4.822625,9.562394,5.213161,13.220644,5.330401,6.590039,6.423038,6.419294,6.627119,...,7.159241,7.011830,4.020500,9.456660,5.257900,11.032722,5.488203,8.042470,7.606461,9.989334
4,GSM1026997_00-129BM_3.CEL,5.788213,8.869127,7.369893,13.634581,5.194392,6.393579,6.085670,6.428252,6.524573,...,7.083558,6.771828,5.919560,8.619094,5.676901,10.468684,4.899533,8.625207,7.569196,8.984389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,GSM1027056_D10CV.CEL,7.446580,9.368198,8.185807,13.445603,5.561658,6.270590,6.727135,6.578518,6.703231,...,7.098490,7.070867,5.388457,7.811394,5.773009,10.703419,5.516487,8.609645,8.268249,9.292362
64,GSM1027057_D17CM.CEL,5.459205,8.978038,6.104034,13.244360,5.262246,5.834211,6.535886,6.823762,6.787169,...,7.203564,7.159263,5.371347,8.298846,5.484968,10.292255,5.234787,8.101849,7.973776,9.498614
65,GSM1027058_D18CM.CEL,5.284065,9.142073,5.835966,13.269262,5.197876,6.041725,6.601121,6.315047,6.560862,...,7.190959,6.737819,4.591442,8.778146,5.387309,10.148537,5.658559,8.175294,7.663093,9.304413
66,GSM1027059_D1CV.CEL,7.358534,8.827518,8.282384,13.421351,4.959862,5.608704,6.304388,6.514108,6.527713,...,7.132263,7.391099,5.667911,7.030987,5.690151,10.113436,5.607732,8.861562,8.074523,8.872957


Mapping GSE21942

In [81]:
gse21942 = pd.read_csv('gse21942_norm.csv')
gse21942.columns = ['SampleID'] + list([probeMapDef[2][probeId] if probeId in probeMapDef[2] else None for probeId in gse21942.columns[1:]])
gse21942 = gse21942.drop(columns=[col for col in gse21942.columns if col is None])
gse21942 = changeNameDuplicates(gse21942)
gse21942 = aggregateProbes(gse21942)
gse21942.columns = gse21942.columns.str.split('_').str[0]

print(gse21942.columns.is_unique)
print(gse21942.columns.str.contains("_").any())

gse21942.to_csv(path+'/MappedNormalized/gse21942_mapped.csv', index=False)

gse21942

True
False


,SampleID,ENSG00000204580,ENSG00000049541,ENSG00000173110,ENSG00000125618,ENSG00000048545,ENSG00000182179,ENSG00000126351,ENSG00000070778,ENSG00000271503,...,ENSG00000173093,ENSG00000229656,ENSG00000246582,ENSG00000188897,ENSG00000128310,ENSG00000259291,ENSG00000205307,ENSG00000205084,ENSG00000234857,ENSG00000260735
0,GSM545818.CEL,6.715101,7.277721,9.017146,7.699178,6.515594,8.806566,7.343849,4.766834,12.470949,...,4.297832,4.906896,4.488447,5.665330,6.894686,6.690123,6.773862,3.035751,4.853029,6.783218
1,GSM545819.CEL,6.785745,7.369654,9.057797,7.434026,6.424909,8.772475,7.331029,4.304934,11.895876,...,4.281347,4.968197,4.810457,5.825328,6.842709,6.183091,7.167958,2.999182,4.711712,6.978498
2,GSM545820.CEL,7.052194,7.224084,9.031111,7.714347,6.424909,8.726472,7.310814,4.536488,11.464701,...,4.353803,4.546266,4.431380,5.598694,6.840796,6.574699,7.037205,2.850802,4.524578,6.552829
3,GSM545821.CEL,6.747209,7.176329,8.796729,7.443340,6.437856,8.643565,7.287243,4.636765,12.665831,...,4.329372,4.634123,4.312059,5.944063,6.733381,6.952205,7.579859,3.131717,4.419525,6.506585
4,GSM545822.CEL,6.834951,7.324719,8.963759,7.534118,6.424909,8.944224,7.481922,4.673901,12.353911,...,3.964592,4.598256,4.897950,5.899544,6.681757,6.363068,7.049641,2.972100,4.644753,6.563500
5,GSM545823.CEL,7.008971,7.046270,8.963759,7.846924,6.130116,8.690761,7.573521,4.223929,11.705673,...,4.022467,4.802256,4.723809,5.836961,7.081840,6.597797,7.034257,2.964992,5.093793,7.014917
6,GSM545824.CEL,6.436211,7.085381,9.302709,7.733425,6.435940,8.762705,7.320897,4.597931,11.798920,...,4.324761,4.563522,4.618779,5.604495,6.695236,6.424693,6.932918,2.949460,4.914937,6.835873
7,GSM545825.CEL,6.900510,7.109658,8.952611,7.725928,6.223500,8.712779,7.882660,4.760781,12.289033,...,4.174028,4.885089,4.486285,5.538535,7.117175,6.613247,7.053407,2.909885,4.848834,7.160974
8,GSM545826.CEL,6.653279,7.120752,8.849261,7.522826,6.369805,8.726354,7.136013,4.337521,12.588853,...,4.133267,4.690431,4.853234,5.730634,6.807610,6.692435,6.794944,2.918387,4.888311,7.066187
9,GSM545827.CEL,6.928483,7.410144,8.864259,7.696693,6.436511,8.786080,7.485565,4.829820,12.179772,...,4.009007,4.486380,4.383437,5.893030,6.708924,6.470934,7.434113,3.022845,5.032592,6.891590


I Dataset sono stati normalizzati e gli ID delle sonde sono stati trasformati in Gene SYMBOL tramite script in R

In [11]:
os.chdir(path+'/MappedNormalized')

In [16]:
def header(df):
    names = df.columns.values
    names[0] = "SampleID"
    df.columns = names

In [12]:
path = "/SYMBOL"

<h3>GSE41848</h3>

In [ ]:
gse41848 = pd.read_csv('GSE41848_mapped.csv')
header(gse41848)

os.chdir("../../../Dataset/SampleNotation")
sampleNotation41848 = pd.read_csv("GSE41848_sampleClassification.txt", sep='\t', encoding='utf-8')
os.chdir("../../Ricerca"+path+"/MappedNormalized")

def rename(entry):
    patient_splitted = entry.split(',')[0].split(' ')
    patientID = patient_splitted[2] if len(patient_splitted) == 3 else patient_splitted[1]
    visit_splitted = entry.split(',')[1].split(' ')
    isFollowUp = False if 'Baseline' in visit_splitted[1] else True
    if 'MS' in patient_splitted[0]:
        return patientID+'-FollowUp_'+visit_splitted[3] if isFollowUp else patientID
    else:
        return patientID+'-Control-'+'FollowUp_'+visit_splitted[3] if isFollowUp else patientID+'-'+'Control'

sampleNotation41848 = sampleNotation41848.T
columns = ['Title', 'SampleID']
sampleNotation41848.columns = columns
sampleNotation41848.drop(sampleNotation41848.index[0], inplace=True)
sampleNotation41848['Title'] = sampleNotation41848['Title'].apply(rename)
gse41848['SampleID'] = gse41848['SampleID'].str.split('_').str[0]

gse41848 = sampleNotation41848.merge(gse41848, on='SampleID', how='inner')
gse41848['SampleID'] = gse41848['SampleID']+'_'+gse41848['Title'] 
gse41848.drop(columns=['Title'], inplace=True)

print(gse41848.shape)
print(sum(gse41848['SampleID'].str.contains('-Control')))
print(gse41848['SampleID'].values)
print(gse41848.describe()) #OK NUMERICO
gse41848.head()
os.chdir("../Annotated")
gse41848.to_csv("GSE41848_filtered.csv")
os.chdir("../MappedNormalized")


(212, 17025)
79
['GSM1026056_600009.0001' 'GSM1026057_600009.0001-FollowUp_1'
 'GSM1026058_41461.0001' 'GSM1026059_41462.0001' 'GSM1026060_600029.0001'
 'GSM1026061_41485.0001' 'GSM1026062_41497.0001'
 'GSM1026063_41497.0001-FollowUp_1' 'GSM1026064_41497.0001-FollowUp_2'
 'GSM1026065_41509.0001' 'GSM1026066_41509.0001-FollowUp_2'
 'GSM1026067_41509.0001-FollowUp_1' 'GSM1026068_600096.0001'
 'GSM1026069_600096.0001-FollowUp_1' 'GSM1026070_600096.0001-FollowUp_2'
 'GSM1026071_600104.0001' 'GSM1026072_41510.0001'
 'GSM1026073_600130.0001-FollowUp_1' 'GSM1026074_600134.0001'
 'GSM1026075_600134.0001-FollowUp_1' 'GSM1026076_600134.0001-FollowUp_2'
 'GSM1026077_600139.0001' 'GSM1026078_600139.0001-FollowUp_1'
 'GSM1026079_600139.0001-FollowUp_2' 'GSM1026080_600141.0001-FollowUp_1'
 'GSM1026081_41527.0001-FollowUp_2' 'GSM1026082_600153.0001'
 'GSM1026083_600153.0001-FollowUp_1' 'GSM1026084_600153.0001-FollowUp_2'
 'GSM1026085_41293.0001' 'GSM1026086_41293.0001-FollowUp_1'
 'GSM1026087_41293.0

<h3>GSE41849</h3>

In [ ]:
gse41849 = pd.read_csv('GSE41849_mapped.csv')
header(gse41849)

os.chdir("../../../Dataset/SampleNotation")
sampleNotation41849 = pd.read_csv("GSE41849_sampleClassification.txt", sep='\t', encoding='utf-8', header=None)
os.chdir("../../Ricerca"+path+"/MappedNormalized")

def rename(entry):
    patient_splitted = entry.split(',')[0].split(' ')
    patientID = patient_splitted[2] if len(patient_splitted) == 3 else patient_splitted[1]
    visit_splitted = entry.split(',')[1].split(' ')
    isFollowUp = False if 'Baseline' in visit_splitted[1] else True
    if 'MS' in patient_splitted[0]:
        return patientID+'-FollowUp_'+visit_splitted[3] if isFollowUp else patientID
    else:
        return patientID+'-Control-'+'FollowUp_'+visit_splitted[3] if isFollowUp else patientID+'-'+'Control'

sampleNotation41849 = sampleNotation41849.T
columns = ['Title', 'SampleID', 'Gender']
sampleNotation41849.columns = columns
sampleNotation41849.drop(sampleNotation41849.index[0], inplace=True)
sampleNotation41849['Title'] = sampleNotation41849['Title'].apply(rename)
sampleNotation41849['Gender'] = sampleNotation41849['Gender'].apply(lambda x: x.split(' ')[1])
gse41849['SampleID'] = gse41849['SampleID'].str.split('_').str[0]


gse41849 = sampleNotation41849.merge(gse41849, on='SampleID', how='inner')
gse41849['SampleID'] = gse41849['SampleID']+'_'+gse41849['Title'] 
gse41849.drop(columns=['Title'], inplace=True)

print(gse41849.shape)
print(sum(gse41849['SampleID'].str.contains('-Control')))
print(gse41849['SampleID'].values)
print(gse41849.describe()) #OK NUMERICO
gse41849.head()
os.chdir("../Annotated")
gse41849.to_csv("GSE41849_filtered.csv")
os.chdir("../MappedNormalized")

(102, 17026)
46
['GSM1026268_600001.0001' 'GSM1026269_600001.0001-FollowUp_1'
 'GSM1026270_41495.0001' 'GSM1026271_41495.0001-FollowUp_1'
 'GSM1026272_41495.0001-FollowUp_2' 'GSM1026273_600076.0001-FollowUp_2'
 'GSM1026274_600089.0001' 'GSM1026275_600182.0001'
 'GSM1026276_600183.0001' 'GSM1026277_600188.0001'
 'GSM1026278_600188.0001-FollowUp_1' 'GSM1026279_600188.0001-FollowUp_2'
 'GSM1026280_600189.0001-FollowUp_1' 'GSM1026281_600189.0001-FollowUp_2'
 'GSM1026282_41743.0001' 'GSM1026283_41743.0001-FollowUp_1'
 'GSM1026284_41743.0001-FollowUp_2' 'GSM1026285_610039.0001-FollowUp_1'
 'GSM1026286_600229.0001-FollowUp_1' 'GSM1026287_600229.0001-FollowUp_2'
 'GSM1026288_41567.0001' 'GSM1026289_600245.0001'
 'GSM1026290_600245.0001-FollowUp_1' 'GSM1026291_600245.0001-FollowUp_2'
 'GSM1026292_600257.0001' 'GSM1026293_600257.0001-FollowUp_1'
 'GSM1026294_600257.0001-FollowUp_2' 'GSM1026295_600279.0001-FollowUp_2'
 'GSM1026296_600281.0001' 'GSM1026297_600292.0001'
 'GSM1026298_600292.0001-Fol

<h3>GSE146383</h3>

In [ ]:
gse146383 = pd.read_csv("GSE146383_mapped.csv")
header(gse146383)

os.chdir("../../../Dataset/SampleNotation")
sampleNotation146383 = pd.read_csv("GSE146383_sampleClassification.txt", sep='\t', encoding='utf-8')
os.chdir("../../Ricerca"+path+"/MappedNormalized")

sampleNotation146383 = sampleNotation146383.T
sampleNotation146383.columns = sampleNotation146383.iloc[0]
sampleNotation146383.drop(sampleNotation146383.index[0], inplace=True)
sampleNotation146383 = sampleNotation146383[sampleNotation146383['Sample_title'].str.contains('ad|AD', na=False)]

gse146383['SampleID'] = gse146383['SampleID'].str.split('_').str[0]
gse146383 = sampleNotation146383[['Sample_title', 'Sample_geo_accession']].merge(gse146383, right_on='SampleID', left_on='Sample_geo_accession', how='inner')
gse146383.drop(columns=['Sample_geo_accession'], inplace=True)
gse146383['SampleID'] = gse146383['SampleID']+"_"+gse146383['Sample_title']
gse146383.drop(columns=['Sample_title'], inplace=True)

def rename(entry):
    entry_splitted = entry.split('_')
    match = re.findall(r'\s(\w+)', entry_splitted[1])
    isControl = True if 'ADC' in entry else False
    return entry_splitted[0]+'_'+match[0]+'-Control' if isControl else entry_splitted[0]+'_'+match[0]

gse146383['SampleID'] = gse146383['SampleID'].apply(rename)

print(gse146383.shape)
print(sum(gse146383['SampleID'].str.contains('-Control')))
gse146383.head()
print(gse146383['SampleID'].values)
print(gse146383.describe()) #OK NUMERICO
os.chdir("../Annotated")
gse146383.to_csv("GSE146383_filtered.csv")
os.chdir("../MappedNormalized")

(61, 13011)
46
['GSM4379744_7618ADMS212' 'GSM4379746_11283ADMS213'
 'GSM4379750_6495ADMS211' 'GSM4379751_8125ADMS212'
 'GSM4379752_13653ADMS210' 'GSM4379753_1662ADMS213'
 'GSM4379757_10677ADMS210' 'GSM4379758_4956ADMS212'
 'GSM4379759_7675ADMS211' 'GSM4379760_7701ADMS213'
 'GSM4379761_6069ADMS213' 'GSM4379762_7820ADMS212'
 'GSM4379768_12833ADMS211' 'GSM4379769_13546ADMS210'
 'GSM4379770_5219ADMS211' 'GSM4379781_6291PDC212-Control'
 'GSM4379783_5645C108-Control' 'GSM4379784_12912C190-Control'
 'GSM4379785_4491C73-Control' 'GSM4379786_1568-Control'
 'GSM4379787_5368C98-Control' 'GSM4379788_13409NMS203-Control'
 'GSM4379789_1891-Control' 'GSM4379790_3069C55-Control'
 'GSM4379791_1571C44-Control' 'GSM4379792_4489C73-Control'
 'GSM4379793_1566C46-Control' 'GSM4379794_2466C48-Control'
 'GSM4379795_3326C58-Control' 'GSM4379796_4200C69-Control'
 'GSM4379797_12899C189-Control' 'GSM4379798_13408NMS203-Control'
 'GSM4379799_2720C50-Control' 'GSM4379800_3070C56-Control'
 'GSM4379801_2120C46-Contro

<h3>GSE13732</h3>

In [ ]:
# os.chdir('..')
gse13732 = pd.read_csv('GSE13732_mapped.csv')
header(gse13732)

os.chdir("../../../Dataset/SampleNotation")
sampleNotation13732 = pd.read_csv("GSE13732_sampleClassification.txt", sep='\t', encoding='utf-8', header=None)
os.chdir("../../Ricerca"+path+"/MappedNormalized")

def rename(entry):
    patient_splitted = entry.split('_')
    patientID = patient_splitted[-1]
    isFollowUp = False if 'baseline' in patient_splitted[1] else True
    if 'CIS' in patient_splitted[0]:
        return patientID+'-FollowUp_'+patient_splitted[1][-1] if isFollowUp else patientID
    else:
        return patientID+'-Control-'+'FollowUp_'+patient_splitted[1][-1] if isFollowUp else patientID+'-'+'Control'

sampleNotation13732 = sampleNotation13732.T
columns = ['Title', 'SampleID']
sampleNotation13732.columns = columns
sampleNotation13732.drop(sampleNotation13732.index[0], inplace=True)
sampleNotation13732['Title'] = sampleNotation13732['Title'].apply(rename)
# sampleNotation13732['Gender'] = sampleNotation13732['Gender'].apply(lambda x: x.split(' ')[1])

gse13732['SampleID'] = gse13732['SampleID'].str.split('.').str[0]
gse13732 = sampleNotation13732.merge(gse13732, on='SampleID', how='inner')
gse13732['SampleID'] = gse13732['SampleID']+'_'+gse13732['Title'] 
gse13732.drop(columns=['Title'], inplace=True)

print(gse146383.shape)
print(sum(gse13732['SampleID'].str.contains('-Control')))
print(gse13732['SampleID'].values)
print(gse13732.describe()) #OK NUMERICO
gse13732.head()
os.chdir("../Annotated")
gse13732.to_csv("GSE13732_filtered.csv")
os.chdir("../MappedNormalized")

(61, 13011)
40
['GSM345077_cis001' 'GSM345078_cis002' 'GSM345079_cis003'
 'GSM345080_cis004' 'GSM345081_cis005' 'GSM345082_cis006'
 'GSM345083_cis007' 'GSM345084_cis008' 'GSM345085_cis009'
 'GSM345086_cis010' 'GSM345087_cis010b' 'GSM345088_cis011'
 'GSM345089_cis012' 'GSM345090_cis013' 'GSM345091_cis013b'
 'GSM345092_cis014' 'GSM345093_cis015' 'GSM345094_2' 'GSM345095_cis017'
 'GSM345096_cis018' 'GSM345097_cis020' 'GSM345098_cis021'
 'GSM345099_cis022' 'GSM345100_cis024' 'GSM345101_cis025'
 'GSM345102_cis026' 'GSM345103_cis027' 'GSM345104_cis028'
 'GSM345105_cis029' 'GSM345106_cis030' 'GSM345107_cis032'
 'GSM345108_cis033' 'GSM345109_cis034' 'GSM345110_cis037'
 'GSM345111_cis038' 'GSM345112_cis039' 'GSM345113_cis040'
 'GSM345114_cis041' 'GSM345115_cis043' 'GSM345116_ctr001-Control'
 'GSM345117_ctr001b-Control' 'GSM345118_ctr002-Control'
 'GSM345119_ctr002b-Control' 'GSM345120_ctr003-Control'
 'GSM345121_ctr004-Control' 'GSM345122_ctr006-Control'
 'GSM345123_ctr007-Control' 'GSM345124_c

<h3>GSE136411</h3>

In [20]:
gse136411 = pd.read_csv("GSE136411_mapped.csv")
header(gse136411)

os.chdir("../../../Dataset/SampleNotation")
sampleNotation136411_1 = pd.read_csv("GSE_136411_6104_sampleClassification.txt", sep='\t', header=None, index_col=False)
os.chdir("../../Ricerca"+path+"/MappedNormalized")

sampleNotation136411_1 = sampleNotation136411_1.T
sampleNotation136411_1.drop(sampleNotation136411_1.index[0], inplace=True)
sampleNotation136411_1.columns = ['SampleID', 'Accession', 'Title']

os.chdir("../../../Dataset/SampleNotation")
sampleNotation136411_2 = pd.read_csv("GSE_136411_10558_sampleClassification.txt", sep='\t', header=None, index_col=False)
os.chdir("../../Ricerca"+path+"/MappedNormalized")

sampleNotation136411_2 = sampleNotation136411_2.T
sampleNotation136411_2.drop(sampleNotation136411_2.index[0], inplace=True)
sampleNotation136411_2.columns = ['SampleID', 'Accession', 'Title']

sampleNotation136411 = pd.concat([sampleNotation136411_1, sampleNotation136411_2], axis=0)
sampleNotation136411.reset_index(drop=True, inplace=True)

delete = []
for i in range(287, 314):
    delete.append("Sample"+str(i))

gse136411 = gse136411[~gse136411['SampleID'].isin(delete)]

columns = gse136411.columns.values[1:]
gse136411[columns] = gse136411[columns].replace(',', '.', regex=True).astype(float)
gse136411 = sampleNotation136411.merge(gse136411, on='SampleID', how='inner')
gse136411['SampleID'] = gse136411['Accession']+'_'+gse136411['Title']
gse136411.drop(columns=['Title', 'Accession'], inplace=True)

def rename(entry):
    entry_splitted = entry.split('_')
    match = re.findall(r'rep(\d+)', entry_splitted[3])
    followUp = entry_splitted[3].split('.')
    isControl = True if 'HC' in entry_splitted[2] else False
    if len(followUp) == 1:
        return '4-'+entry_splitted[0]+'_'+match[0]+'-Control' if isControl else entry_splitted[0]+'_'+match[0]
    else:
        return '4-'+entry_splitted[0]+'_'+match[0]+'-Control'+'-FollowUp_'+followUp[1] if isControl else entry_splitted[0]+'_'+match[0]+'-FollowUp_'+followUp[1]

gse136411['SampleID'] = gse136411['SampleID'].apply(rename)

#Gli ID dei pazienti sono errati, chiamare questa funzione dopo tutte le altre per sistemarli
def rename2(dataset, sampleNotation):
    sampleID = dataset['SampleID']
    for i in range(len(sampleID)):
        if sampleID.iloc[i].startswith('4'):
            oldID = sampleID.iloc[i]
            sampleID.iloc[i] = sampleID.iloc[i].split('-')[1].split('_')[0]
            newID = ''
            if sampleID.iloc[i] in sampleNotation['Accession'].values:
                sampleNot = sampleNotation[sampleNotation['Accession'] == sampleID.iloc[i]]
                match = re.search(r'\d+', sampleNot['SampleID'].iloc[0].split('.')[0].split('_')[0])
                newID = sampleID.iloc[i]+'_'+match.group(0)

                if 'HC' in sampleNot['Title'].iloc[0]:
                    newID += '-Control'

                if sampleNot['Title'].iloc[0].find('.') != -1:
                    newID += '-FollowUp_'+sampleNot['Title'].iloc[0].split('.')[1]

                sampleID.iloc[i] = newID

rename2(gse136411, sampleNotation136411)

print(gse136411.shape)
print(sum(gse136411['SampleID'].str.contains('-Control')))
gse136411.head()
print(gse136411['SampleID'].values)
print(gse136411.describe()) #OK 
os.chdir("../Annotated")
gse136411.to_csv("GSE136411_filtered.csv")
os.chdir("../MappedNormalized")

C:\Users\samue\AppData\Local\Temp\ipykernel_1072\4007201660.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleID.iloc[i] = sampleID.iloc[i].split('-')[1].split('_')[0]
C:\Users\samue\AppData\Local\Temp\ipykernel_1072\4007201660.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleID.iloc[i] = newID
C:\Users\samue\AppData\Local\Temp\ipykernel_1072\4007201660.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleID.il

(309, 9087)
67
['GSM4048323_1' 'GSM4048324_2' 'GSM4048325_3' 'GSM4048326_3-FollowUp_1'
 'GSM4048327_4' 'GSM4048328_5' 'GSM4048329_5-FollowUp_1' 'GSM4048330_6'
 'GSM4048331_6-FollowUp_1' 'GSM4048332_7' 'GSM4048333_8' 'GSM4048334_9'
 'GSM4048335_10' 'GSM4048336_11' 'GSM4048337_12' 'GSM4048338_13'
 'GSM4048339_14' 'GSM4048340_15' 'GSM4048341_16' 'GSM4048342_17'
 'GSM4048343_18' 'GSM4048344_19' 'GSM4048345_20' 'GSM4048346_21'
 'GSM4048347_22' 'GSM4048348_23' 'GSM4048349_24' 'GSM4048350_25'
 'GSM4048351_26' 'GSM4048352_27' 'GSM4048353_28' 'GSM4048354_29'
 'GSM4048355_30' 'GSM4048356_31' 'GSM4048357_32' 'GSM4048358_33'
 'GSM4048359_34' 'GSM4048360_35' 'GSM4048361_36' 'GSM4048362_37'
 'GSM4048363_38' 'GSM4048364_39' 'GSM4048365_40' 'GSM4048366_41'
 'GSM4048367_42' 'GSM4048374_49' 'GSM4048375_50' 'GSM4048376_51'
 'GSM4048377_52' 'GSM4048383_1' 'GSM4048386_2' 'GSM4048387_3'
 'GSM4048388_4' 'GSM4048389_5' 'GSM4048390_6' 'GSM4048391_7'
 'GSM4048392_8' 'GSM4048393_9' 'GSM4048394_10' 'GSM4048395_11

<h3>GSE17048</h3>

In [ ]:
gse17048= pd.read_csv("GSE17048_mapped.csv")
header(gse17048)

os.chdir("../../../Dataset/SampleNotation")
sampleNotation17048 = pd.read_csv("GSE17048_sampleClassification.txt", sep='\t', encoding='utf-8', header=None).T
os.chdir("../../Ricerca"+path+"/MappedNormalized")

sampleNotation17048.columns = ['GSEID', 'Title']
sampleNotation17048.drop(sampleNotation17048.index[0], inplace=True)
sampleNotation17048['SampleID'] = sampleNotation17048['Title'].str.split(', ').str[-1]

def rename(entry):
    patient_splitted = entry.split(', ')
    patientID = patient_splitted[-1]
    if 'control' not in patient_splitted[0]:
        return patientID
    else:
        return  patientID+'-'+'Control'

sampleNotation17048['Title'] = sampleNotation17048['Title'].apply(rename)

gse17048 = sampleNotation17048.merge(gse17048, on='SampleID', how='inner')
gse17048['SampleID'] = gse17048['GSEID']+'_'+gse17048['Title'] 
gse17048.drop(columns=['Title', 'GSEID'], inplace=True)

print(gse17048.shape)
print(sum(gse17048['SampleID'].str.contains('-Control')))
gse17048.head()
os.chdir("../Annotated")
gse17048.to_csv("GSE17048_filtered.csv")
os.chdir("../MappedNormalized")

(144, 19037)
45


GSE41890

In [23]:
gse41890 = pd.read_csv('GSE41890_mapped.csv')
header(gse41890)

os.chdir("../../../Dataset/SampleNotation")
sampleNotation41890 = pd.read_csv("GSE41890_sampleClassification.txt", sep='\t', encoding='utf-8')
os.chdir("../../Ricerca"+path+"/MappedNormalized")

def rename(entry):
    patient_splitted = entry.split(' ')
    patientID = patient_splitted[-1][1:-1].replace('-', '').replace('_', '')
    if 'patient' in patient_splitted[0]:
        return patientID if 'N' not in patientID else patientID.replace('N', '')+'-FollowUp_1'
    else:
        return patientID+'-Control'

sampleNotation41890 = sampleNotation41890.T
sampleNotation41890.insert(0, "Title", sampleNotation41890.index)
sampleNotation41890.reset_index(drop=True, inplace=True)
columns = ['Title', 'SampleID']
sampleNotation41890.columns = columns
sampleNotation41890.drop(sampleNotation41890.index[0], inplace=True)
sampleNotation41890['Title'] = sampleNotation41890['Title'].apply(rename)

gse41890['SampleID'] = gse41890['SampleID'].str.split('_').str[0]

gse41890 = sampleNotation41890.merge(gse41890, on='SampleID', how='inner')
gse41890['SampleID'] = gse41890['SampleID']+'_'+gse41890['Title'] 
gse41890.drop(columns=['Title'], inplace=True)

print(gse41890.shape)
print(sum(gse41890['SampleID'].str.contains('-Control')))
print(gse41890['SampleID'].values)
print(gse41890.describe()) #OK NUMERICO
gse41890.head()
os.chdir("../Annotated")
gse41890.to_csv("GSE41890_filtered.csv")
os.chdir("../MappedNormalized")

(68, 19772)
24
['GSM1026993_00024BM' 'GSM1026994_00024BM-FollowUp_1' 'GSM1026995_00096BV'
 'GSM1026996_00096BV-FollowUp_1' 'GSM1026997_00129BM3'
 'GSM1026998_00129BM3-FollowUp_1' 'GSM1026999_04074BV22'
 'GSM1027000_04074BV22-FollowUp_1' 'GSM1027001_05004BM'
 'GSM1027002_05004BM-FollowUp_1' 'GSM1027003_05007BM'
 'GSM1027004_05007BM-FollowUp_1' 'GSM1027005_06185BM'
 'GSM1027006_06185BM-FollowUp_1' 'GSM1027007_07254BM2'
 'GSM1027008_07254BM-FollowUp_1' 'GSM1027009_07309BV'
 'GSM1027010_07309BV-FollowUp_1' 'GSM1027011_08111BV'
 'GSM1027012_08111BV-FollowUp_1' 'GSM1027013_08377BM3'
 'GSM1027014_08377BM3-FollowUp_1' 'GSM1027015_08584CV-Control'
 'GSM1027016_08588CM-Control' 'GSM1027017_08647CV-Control'
 'GSM1027018_08650CM-Control' 'GSM1027019_08659CM-Control'
 'GSM1027020_08759CV-Control' 'GSM1027021_091203CM-Control'
 'GSM1027022_09152BM' 'GSM1027023_09152BM-FollowUp_1'
 'GSM1027024_09221CV-Control' 'GSM1027025_09394BV3'
 'GSM1027026_09394BV3-FollowUp_1' 'GSM1027027_09475BV4'
 'GSM1027028_

GSE21942

In [86]:
gse21942 = pd.read_csv('GSE21942_mapped.csv')
header(gse21942)

os.chdir("../../../Dataset/SampleNotation")
sampleNotation21942 = pd.read_csv("GSE21942_sampleClassification.txt", sep='\t', encoding='utf-8')
os.chdir("../../Ricerca"+path+"/MappedNormalized")

def rename(entry):
    patient_splitted = entry.split(',')
    patientID = patient_splitted[0].split(' ')[-1]
    if 'MS' in patient_splitted[0]:
        return "M"+patientID if 'replicate 2' not in patient_splitted[-1] else "M"+patientID+'-FollowUp_1'
    else:
        return "C"+patientID+'-Control'

sampleNotation21942 = sampleNotation21942.T
sampleNotation21942.insert(0, "Title", sampleNotation21942.index)
sampleNotation21942.reset_index(drop=True, inplace=True)
columns = ['Title', 'SampleID']
sampleNotation21942.columns = columns
sampleNotation21942.drop(sampleNotation21942.index[0], inplace=True)
sampleNotation21942['Title'] = sampleNotation21942['Title'].apply(rename)

sampleNotation21942

gse21942['SampleID'] = gse21942['SampleID'].str.split('.').str[0]

gse21942 = sampleNotation21942.merge(gse21942, on='SampleID', how='inner')
gse21942['SampleID'] = gse21942['SampleID']+'_'+gse21942['Title'] 
gse21942.drop(columns=['Title'], inplace=True)

print(gse21942.shape)
print(sum(gse21942['SampleID'].str.contains('-Control')))
print(gse21942['SampleID'].values)
print(gse21942.describe()) #OK NUMERICO
gse21942.head()
os.chdir("../Annotated")
gse21942.to_csv("GSE21942_filtered.csv")
os.chdir("../MappedNormalized")

(29, 20143)
15
['GSM545818_C1-Control' 'GSM545819_C2-Control' 'GSM545820_C3-Control'
 'GSM545821_C4-Control' 'GSM545822_C5-Control' 'GSM545823_C6-Control'
 'GSM545824_C7-Control' 'GSM545825_C8-Control' 'GSM545826_C9-Control'
 'GSM545827_C10-Control' 'GSM545828_C11-Control' 'GSM545829_C12-Control'
 'GSM545830_C13-Control' 'GSM545831_C14-Control' 'GSM545832_C15-Control'
 'GSM545833_M1' 'GSM545834_M2' 'GSM545835_M3' 'GSM545836_M4'
 'GSM545837_M5' 'GSM545838_M6' 'GSM545839_M7' 'GSM545840_M8'
 'GSM545841_M9' 'GSM545842_M10' 'GSM545843_M11' 'GSM545844_M11-FollowUp_1'
 'GSM545845_M12' 'GSM545846_M12-FollowUp_1']
       ENSG00000204580  ENSG00000049541  ENSG00000173110  ENSG00000125618  \
count        29.000000        29.000000        29.000000        29.000000   
mean          6.937049         7.098944         8.972133         7.627513   
std           0.200896         0.192880         0.314516         0.113241   
min           6.436211         6.779245         8.256087         7.434026   
25